# Agentes com Memória

Temos um assistente de email que usa um roteador para triagem de emails e então passa o email para o agente gerar uma resposta. Também o avaliamos e adicionamos humano-no-loop (HITL). Mas e se nosso assistente pudesse **aprender** com as interações para melhorar ao longo do tempo? Isso é onde a **memória** entra.

![overview-img](img/overview_memory.png)

A memória no LangGraph opera em dois níveis distintos:

**Memória Com Escopo de Thread (Curto prazo)** opera dentro dos limites de um único thread de conversa. É gerenciada automaticamente como parte do estado do grafo e persistida através de checkpoints com escopo de thread. Este tipo de memória retém histórico de conversa, arquivos enviados, documentos recuperados e outros artefatos gerados durante a interação. Pense nisso como a memória de trabalho que mantém contexto dentro de uma conversa específica, permitindo que o agente referencie mensagens ou ações anteriores sem começar do zero a cada vez.

**Memória Através de Threads (Longo prazo)** se estende além de conversas individuais, criando uma base de conhecimento persistente que abrange múltiplas sessões. Esta memória é armazenada como documentos JSON em um armazenamento de memória, organizada por namespaces (como pastas) e chaves distintas (como nomes de arquivo). Diferente da memória com escopo de thread, esta informação persiste mesmo após as conversas terminarem, permitindo que o sistema se lembre de preferências do usuário, decisões passadas e conhecimento acumulado. Isso é o que permite que um agente verdadeiramente aprenda e se adapte ao longo do tempo, em vez de tratar cada interação como isolada.

#### Carregar Variáveis de Ambiente

In [ ]:
from dotenv import load_dotenv
load_dotenv("../.env")

## Memória no LangGraph

### Memória Com Escopo de Thread e Através de Threads

Primeiro, vale a pena explicar como [a memória funciona no LangGraph](https://langchain-ai.github.io/langgraph/concepts/memory/). LangGraph oferece dois tipos principais de memória:

**Memória Com Escopo de Thread (Curto prazo)** opera dentro dos limites de um único thread de conversa. É gerenciada automaticamente como parte do estado do grafo e persistida através de checkpoints com escopo de thread. Este tipo de memória retém histórico de conversa, arquivos enviados, documentos recuperados e outros artefatos gerados durante a interação. Pense nisso como a memória de trabalho que mantém contexto dentro de uma conversa específica, permitindo que o agente referencie mensagens ou ações anteriores sem começar do zero a cada vez.

**Memória Através de Threads (Longo prazo)** se estende além de conversas individuais, criando uma base de conhecimento persistente que abrange múltiplas sessões. Esta memória é armazenada como documentos JSON em um armazenamento de memória, organizada por namespaces (como pastas) e chaves distintas (como nomes de arquivo). Diferente da memória com escopo de thread, esta informação persiste mesmo após as conversas terminarem, permitindo que o sistema se lembre de preferências do usuário, decisões passadas e conhecimento acumulado. Isso é o que permite que um agente verdadeiramente aprenda e se adapte ao longo do tempo, em vez de tratar cada interação como isolada.

Existem diferentes opções de armazenamento para memória através de threads:

1. **Na Memória (ex. notebooks)**:
   - Usa `from langgraph.store.memory import InMemoryStore`
   - Simples e rápido para testes e desenvolvimento local
2. **Desenvolvimento Local com `langgraph dev`**:
   - Similar ao InMemoryStore mas com pseudo-persistência
3. **LangGraph Platform ou Deployments de Produção**:
   - Para deployments hospedados, você tem acesso a um armazenamento de produção

Vamos usar o `InMemoryStore` aqui no notebook!

In [ ]:
from langgraph.store.memory import InMemoryStore
in_memory_store = InMemoryStore()

As memórias são organizadas por namespace usando uma tupla, que neste exemplo específico será (`<user_id>`, "memories"). O namespace pode ter qualquer comprimento e representar qualquer coisa, não precisa ser específico do usuário.

In [ ]:
user_id = "1"
namespace_for_memory = (user_id, "memories")

Usamos o método `store.put` para salvar memórias em nosso namespace no armazenamento. Quando fazemos isso, especificamos o namespace, como definido acima, e um par chave-valor para a memória: a chave é simplesmente um identificador único para a memória (memory_id) e o valor (um dicionário) é a própria memória.

In [ ]:
import uuid
memory_id = str(uuid.uuid4())
memory = {"food_preference" : "I like pizza"}
in_memory_store.put(namespace_for_memory, memory_id, memory)

Podemos ler as memórias em nosso namespace usando o método `store.search`, que retornará todas as memórias para um usuário específico como uma lista. A memória mais recente é a última da lista. Cada tipo de memória é uma classe Python (`Item`) com certos atributos. Podemos acessá-la como um dicionário convertendo via `.dict`. Os atributos que possui são mostrados abaixo, mas o mais importante geralmente é `value`.

In [ ]:
memories = in_memory_store.search(namespace_for_memory)
memories[-1].dict()

Para usar isso em um grafo, tudo que precisamos fazer é compilar o grafo com o store:

In [ ]:
# We need this because we want to enable threads (conversations)
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()
# We need this because we want to enable across-thread memory
from langgraph.store.memory import InMemoryStore
in_memory_store = InMemoryStore()
# Compile the graph with the checkpointer and store
# graph = graph.compile(checkpointer=checkpointer, store=in_memory_store)

Vamos pegar nosso grafo com HITL e adicionar memória a ele. Isso será muito similar ao que tínhamos anteriormente. Simplesmente atualizaremos a memória no store quando recebermos feedback do usuário.

A principal diferença será adicionar um [LangGraph Store](https://langchain-ai.github.io/langgraph/concepts/memory/#long-term-memory) para persistir as memórias.

## Memória

Vamos implementar memória que captura:

In [ ]:
%load_ext autoreload
%autoreload 2

from typing import Literal
from datetime import datetime
from pydantic import BaseModel, Field

from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

from langgraph.graph import StateGraph, START, END
from langgraph.store.base import BaseStore
from langgraph.types import interrupt, Command

from email_assistant.prompts import triage_system_prompt, triage_user_prompt, agent_system_prompt_hitl_memory, default_triage_instructions, default_background, default_response_preferences, default_cal_preferences, MEMORY_UPDATE_INSTRUCTIONS, MEMORY_UPDATE_INSTRUCTIONS_REINFORCEMENT
from email_assistant.tools.default.prompt_templates import HITL_MEMORY_TOOLS_PROMPT
from email_assistant.schemas import State, RouterSchema, StateInput
from email_assistant.utils import parse_email, format_for_display, format_email_markdown

# Agent tools
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}' and content: {content}"

@tool
def schedule_meeting(
    attendees: list[str], subject: str, duration_minutes: int, preferred_day: datetime, start_time: int
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"Meeting '{subject}' scheduled on {date_str} at {start_time} for {duration_minutes} minutes with {len(attendees)} attendees"

@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

@tool
class Question(BaseModel):
      """Question to ask user."""
      content: str

@tool
class Done(BaseModel):
      """E-mail has been sent."""
      done: bool

# All tools available to the agent
tools = [
    write_email,
    schedule_meeting,
    check_calendar_availability,
    Question,
    Done
]

tools_by_name = {tool.name: tool for tool in tools}

# Initialize the LLM for use with router / structured output
llm = init_chat_model("gemini-2.5-flash", model_provider="google-genai", temperature=0.0)
llm_router = llm.with_structured_output(RouterSchema)

# Initialize the LLM, enforcing tool use (of any available tools) for agent
llm = init_chat_model("gemini-2.5-flash", model_provider="google-genai", temperature=0.0)
llm_with_tools = llm.bind_tools(tools, tool_choice="any")

Agora, esta é a parte crítica! Atualmente não capturamos nenhum feedback do usuário em nosso grafo.

### Gerenciamento de Memória

O que *queremos* fazer é bastante direto: queremos adicionar o feedback ao `Store` de memória. Se compilarmos nosso grafo com o `Store`, podemos acessá-lo em qualquer nó. Então vamos modificar o `interrupt_handler` para salvar o feedback do usuário no `Store`.

In [ ]:
from rich.markdown import Markdown
Markdown(default_triage_instructions)

In [ ]:
Markdown(default_cal_preferences)

In [ ]:
Markdown(default_response_preferences)

In [ ]:
def get_memory(store, namespace, default_content=None):
    """Get memory from the store or initialize with default if it doesn't exist.

    Args:
        store: LangGraph BaseStore instance to search for existing memory
        namespace: Tuple defining the memory namespace, e.g. ("email_assistant", "triage_preferences")
        default_content: Default content to use if memory doesn't exist

    Returns:
        str: The content of the memory profile, either from existing memory or the default
    """
    # Search for existing memory with namespace and key
    user_preferences = store.get(namespace, "user_preferences")

    # If memory exists, return its content (the value)
    if user_preferences:
        return user_preferences.value

    # If memory doesn't exist, add it to the store and return the default content
    else:
        # Namespace, key, value
        store.put(namespace, "user_preferences", default_content)
        user_preferences = default_content

    # Return the default content
    return user_preferences

Para 2) atualizar memória, podemos usar alguns truques do [guia de prompting GPT-4.1](https://cookbook.openai.com/examples/gpt4-1_prompting_guide) para nos ajudar a atualizar a memória:

1. **Instruções detalhadas**: Fornecer instruções específicas sobre como atualizar preferências
2. **Exemplos**: Mostrar exemplos de atualizações de memória bem formatadas  
3. **Reforço**: Usar lembretes para garantir que as instruções sejam seguidas

In [ ]:

Markdown(MEMORY_UPDATE_INSTRUCTIONS)

In [ ]:
Markdown(MEMORY_UPDATE_INSTRUCTIONS_REINFORCEMENT)

In [ ]:
class UserPreferences(BaseModel):
    """Updated user preferences based on user's feedback."""
    chain_of_thought: str = Field(description="Reasoning about which user preferences need to add / update if required")
    user_preferences: str = Field(description="Updated user preferences")

def update_memory(store, namespace, messages):
    """Update memory profile in the store.

    Args:
        store: LangGraph BaseStore instance to update memory
        namespace: Tuple defining the memory namespace, e.g. ("email_assistant", "triage_preferences")
        messages: List of messages to update the memory with
    """

    # Get the existing memory
    user_preferences = store.get(namespace, "user_preferences")

    # Update the memory
    llm = init_chat_model("gemini-2.5-flash", model_provider="google-genai", temperature=0.0).with_structured_output(UserPreferences)
    result = llm.invoke(
        [
            {"role": "system", "content": MEMORY_UPDATE_INSTRUCTIONS.format(current_profile=user_preferences.value, namespace=namespace)},
        ] + messages
    )

    # Save the updated memory to the store
    store.put(namespace, "user_preferences", result.user_preferences)

Configuramos o roteador de triagem como tínhamos antes, com uma pequena mudança

In [ ]:
def triage_router(state: State, store: BaseStore) -> Command[Literal["triage_interrupt_handler", "response_agent", "__end__"]]:
    """Analyze email content to decide if we should respond, notify, or ignore.

    The triage step prevents the assistant from wasting time on:
    - Marketing emails and spam
    - Company-wide announcements
    - Messages meant for other teams
    """
    # Parse the email input
    author, to, subject, email_thread = parse_email(state["email_input"])
    user_prompt = triage_user_prompt.format(
        author=author, to=to, subject=subject, email_thread=email_thread
    )

    # Create email markdown for Agent Inbox in case of notification
    email_markdown = format_email_markdown(subject, author, to, email_thread)

    # Search for existing triage_preferences memory
    triage_instructions = get_memory(store, ("email_assistant", "triage_preferences"), default_triage_instructions)

    # Format system prompt with background and triage instructions
    system_prompt = triage_system_prompt.format(
        background=default_background,
        triage_instructions=triage_instructions,
    )

    # Run the router LLM
    result = llm_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    # Decision
    classification = result.classification

    # Process the classification decision
    if classification == "respond":
        print("📧 Classificação: RESPONDER - Este email requer uma resposta")
        # Next node
        goto = "response_agent"
        # Update the state
        update = {
            "classification_decision": result.classification,
            "messages": [{"role": "user",
                            "content": f"Respond to the email: {email_markdown}"
                        }],
        }

    elif classification == "ignore":
        print("🚫 Classificação: IGNORAR - Este email pode ser ignorado")

        # Next node
        goto = END
        # Update the state
        update = {
            "classification_decision": classification,
        }

    elif classification == "notify":
        print("🔔 Classificação: NOTIFICAR - Este email contém informações importantes")

        # Next node
        goto = "triage_interrupt_handler"
        # Update the state
        update = {
            "classification_decision": classification,
        }

    else:
        raise ValueError(f"Invalid classification: {classification}")

    return Command(goto=goto, update=update)


Precisamos apenas fazer uma pequena alteração no manipulador de interrupção para atualizar a memória quando o usuário fornecer feedback.

In [ ]:
def triage_interrupt_handler(state: State, store: BaseStore) -> Command[Literal["response_agent", "__end__"]]:
    """Handles interrupts from the triage step"""

    # Parse the email input
    author, to, subject, email_thread = parse_email(state["email_input"])

    # Create email markdown for Agent Inbox in case of notification
    email_markdown = format_email_markdown(subject, author, to, email_thread)

    # Create messages
    messages = [{"role": "user",
                "content": f"Email to notify user about: {email_markdown}"
                }]

    # Create interrupt for Agent Inbox
    request = {
        "action_request": {
            "action": f"Email Assistant: {state['classification_decision']}",
            "args": {}
        },
        "config": {
            "allow_ignore": True,
            "allow_respond": True,
            "allow_edit": False,
            "allow_accept": False,
        },
        # Email to show in Agent Inbox
        "description": email_markdown,
    }

    # Send to Agent Inbox and wait for response
    response = interrupt([request])[0]

    # If user provides feedback, go to response agent and use feedback to respond to email
    if response["type"] == "response":
        # Add feedback to messages
        user_input = response["args"]
        messages.append({"role": "user",
                        "content": f"User wants to reply to the email. Use this feedback to respond: {user_input}"
                        })
        # This is new: update triage_preferences with feedback
        update_memory(store, ("email_assistant", "triage_preferences"), [{
            "role": "user",
            "content": f"The user decided to respond to the email, so update the triage preferences to capture this."
        }] + messages)

        goto = "response_agent"

    # If user ignores email, go to END
    elif response["type"] == "ignore":
        # Make note of the user's decision to ignore the email
        messages.append({"role": "user",
                        "content": f"The user decided to ignore the email even though it was classified as notify. Update triage preferences to capture this."
                        })
        # This is new: triage_preferences with feedback
        update_memory(store, ("email_assistant", "triage_preferences"), messages)
        goto = END

    # Catch all other responses
    else:
        raise ValueError(f"Invalid response: {response}")

    # Update the state
    update = {
        "messages": messages,
    }

    return Command(goto=goto, update=update)

Agora que temos os gerenciadores de memória configurados, podemos usar as preferências armazenadas ao gerar respostas.

In [ ]:
def llm_call(state: State, store: BaseStore):
    """LLM decides whether to call a tool or not"""

    # Search for existing cal_preferences memory
    cal_preferences = get_memory(store, ("email_assistant", "cal_preferences"), default_cal_preferences)

    # Search for existing response_preferences memory
    response_preferences = get_memory(store, ("email_assistant", "response_preferences"), default_response_preferences)

    return {
        "messages": [
            llm_with_tools.invoke(
                [
                    {"role": "system", "content": agent_system_prompt_hitl_memory.format(tools_prompt=HITL_MEMORY_TOOLS_PROMPT,
                                                                                         background=default_background,
                                                                                         response_preferences=response_preferences,
                                                                                         cal_preferences=cal_preferences)}
                ]
                + state["messages"]
            )
        ]
    }

### Integração de Memória no Manipulador de Interrupção

Da mesma forma, adicionaremos memória ao manipulador de interrupção!

In [ ]:
def interrupt_handler(state: State, store: BaseStore) -> Command[Literal["llm_call", "__end__"]]:
    """Creates an interrupt for human review of tool calls"""

    # Store messages
    result = []

    # Go to the LLM call node next
    goto = "llm_call"

    # Iterate over the tool calls in the last message
    for tool_call in state["messages"][-1].tool_calls:

        # Allowed tools for HITL
        hitl_tools = ["write_email", "schedule_meeting", "Question"]

        # If tool is not in our HITL list, execute it directly without interruption
        if tool_call["name"] not in hitl_tools:

            # Execute tool without interruption
            tool = tools_by_name[tool_call["name"]]
            observation = tool.invoke(tool_call["args"])
            result.append({"role": "tool", "content": observation, "tool_call_id": tool_call["id"]})
            continue

        # Get original email from email_input in state
        email_input = state["email_input"]
        author, to, subject, email_thread = parse_email(email_input)
        original_email_markdown = format_email_markdown(subject, author, to, email_thread)

        # Format tool call for display and prepend the original email
        tool_display = format_for_display(tool_call)
        description = original_email_markdown + tool_display

        # Configure what actions are allowed in Agent Inbox
        if tool_call["name"] == "write_email":
            config = {
                "allow_ignore": True,
                "allow_respond": True,
                "allow_edit": True,
                "allow_accept": True,
            }
        elif tool_call["name"] == "schedule_meeting":
            config = {
                "allow_ignore": True,
                "allow_respond": True,
                "allow_edit": True,
                "allow_accept": True,
            }
        elif tool_call["name"] == "Question":
            config = {
                "allow_ignore": True,
                "allow_respond": True,
                "allow_edit": False,
                "allow_accept": False,
            }
        else:
            raise ValueError(f"Invalid tool call: {tool_call['name']}")

        # Create the interrupt request
        request = {
            "action_request": {
                "action": tool_call["name"],
                "args": tool_call["args"]
            },
            "config": config,
            "description": description,
        }

        # Send to Agent Inbox and wait for response
        response = interrupt([request])[0]

        # Handle the responses
        if response["type"] == "accept":

            # Execute the tool with original args
            tool = tools_by_name[tool_call["name"]]
            observation = tool.invoke(tool_call["args"])
            result.append({"role": "tool", "content": observation, "tool_call_id": tool_call["id"]})

        elif response["type"] == "edit":

            # Tool selection
            tool = tools_by_name[tool_call["name"]]
            initial_tool_call = tool_call["args"]

            # Get edited args from Agent Inbox
            edited_args = response["args"]["args"]

            # Update the AI message's tool call with edited content (reference to the message in the state)
            ai_message = state["messages"][-1] # Get the most recent message from the state
            current_id = tool_call["id"] # Store the ID of the tool call being edited

            # Create a new list of tool calls by filtering out the one being edited and adding the updated version
            # This avoids modifying the original list directly (immutable approach)
            updated_tool_calls = [tc for tc in ai_message.tool_calls if tc["id"] != current_id] + [
                {"type": "tool_call", "name": tool_call["name"], "args": edited_args, "id": current_id}
            ]

            # Create a new copy of the message with updated tool calls rather than modifying the original
            # This ensures state immutability and prevents side effects in other parts of the code
            # When we update the messages state key ("messages": result), the add_messages reducer will
            # overwrite existing messages by id and we take advantage of this here to update the tool calls.
            result.append(ai_message.model_copy(update={"tool_calls": updated_tool_calls}))

            # Save feedback in memory and update the write_email tool call with the edited content from Agent Inbox
            if tool_call["name"] == "write_email":

                # Execute the tool with edited args
                observation = tool.invoke(edited_args)

                # Add only the tool response message
                result.append({"role": "tool", "content": observation, "tool_call_id": current_id})

                # This is new: update the memory
                update_memory(store, ("email_assistant", "response_preferences"), [{
                    "role": "user",
                    "content": f"User edited the email response. Here is the initial email generated by the assistant: {initial_tool_call}. Here is the edited email: {edited_args}. Follow all instructions above, and remember: {MEMORY_UPDATE_INSTRUCTIONS_REINFORCEMENT}."
                }])

            # Save feedback in memory and update the schedule_meeting tool call with the edited content from Agent Inbox
            elif tool_call["name"] == "schedule_meeting":

                # Execute the tool with edited args
                observation = tool.invoke(edited_args)

                # Add only the tool response message
                result.append({"role": "tool", "content": observation, "tool_call_id": current_id})

                # This is new: update the memory
                update_memory(store, ("email_assistant", "cal_preferences"), [{
                    "role": "user",
                    "content": f"User edited the calendar invitation. Here is the initial calendar invitation generated by the assistant: {initial_tool_call}. Here is the edited calendar invitation: {edited_args}. Follow all instructions above, and remember: {MEMORY_UPDATE_INSTRUCTIONS_REINFORCEMENT}."
                }])

            # Catch all other tool calls
            else:
                raise ValueError(f"Invalid tool call: {tool_call['name']}")

        elif response["type"] == "ignore":

            if tool_call["name"] == "write_email":
                # Don't execute the tool, and tell the agent how to proceed
                result.append({"role": "tool", "content": "User ignored this email draft. Ignore this email and end the workflow.", "tool_call_id": tool_call["id"]})
                # Go to END
                goto = END
                # This is new: update the memory
                update_memory(store, ("email_assistant", "triage_preferences"), state["messages"] + result + [{
                    "role": "user",
                    "content": f"The user ignored the email draft. That means they did not want to respond to the email. Update the triage preferences to ensure emails of this type are not classified as respond. Follow all instructions above, and remember: {MEMORY_UPDATE_INSTRUCTIONS_REINFORCEMENT}."
                }])

            elif tool_call["name"] == "schedule_meeting":
                # Don't execute the tool, and tell the agent how to proceed
                result.append({"role": "tool", "content": "User ignored this calendar meeting draft. Ignore this email and end the workflow.", "tool_call_id": tool_call["id"]})
                # Go to END
                goto = END
                # This is new: update the memory
                update_memory(store, ("email_assistant", "triage_preferences"), state["messages"] + result + [{
                    "role": "user",
                    "content": f"The user ignored the calendar meeting draft. That means they did not want to schedule a meeting for this email. Update the triage preferences to ensure emails of this type are not classified as respond. Follow all instructions above, and remember: {MEMORY_UPDATE_INSTRUCTIONS_REINFORCEMENT}."
                }])

            elif tool_call["name"] == "Question":
                # Don't execute the tool, and tell the agent how to proceed
                result.append({"role": "tool", "content": "User ignored this question. Ignore this email and end the workflow.", "tool_call_id": tool_call["id"]})
                # Go to END
                goto = END
                # This is new: update the memory
                update_memory(store, ("email_assistant", "triage_preferences"), state["messages"] + result + [{
                    "role": "user",
                    "content": f"The user ignored the Question. That means they did not want to answer the question or deal with this email. Update the triage preferences to ensure emails of this type are not classified as respond. Follow all instructions above, and remember: {MEMORY_UPDATE_INSTRUCTIONS_REINFORCEMENT}."
                }])

            else:
                raise ValueError(f"Invalid tool call: {tool_call['name']}")

        elif response["type"] == "response":
            # User provided feedback
            user_feedback = response["args"]
            if tool_call["name"] == "write_email":
                # Don't execute the tool, and add a message with the user feedback to incorporate into the email
                result.append({"role": "tool", "content": f"User gave feedback, which can we incorporate into the email. Feedback: {user_feedback}", "tool_call_id": tool_call["id"]})
                # This is new: update the memory
                update_memory(store, ("email_assistant", "response_preferences"), state["messages"] + result + [{
                    "role": "user",
                    "content": f"User gave feedback, which we can use to update the response preferences. Follow all instructions above, and remember: {MEMORY_UPDATE_INSTRUCTIONS_REINFORCEMENT}."
                }])

            elif tool_call["name"] == "schedule_meeting":
                # Don't execute the tool, and add a message with the user feedback to incorporate into the email
                result.append({"role": "tool", "content": f"User gave feedback, which can we incorporate into the meeting request. Feedback: {user_feedback}", "tool_call_id": tool_call["id"]})
                # This is new: update the memory
                update_memory(store, ("email_assistant", "cal_preferences"), state["messages"] + result + [{
                    "role": "user",
                    "content": f"User gave feedback, which we can use to update the calendar preferences. Follow all instructions above, and remember: {MEMORY_UPDATE_INSTRUCTIONS_REINFORCEMENT}."
                }])

            elif tool_call["name"] == "Question":
                # Don't execute the tool, and add a message with the user feedback to incorporate into the email
                result.append({"role": "tool", "content": f"User answered the question, which can we can use for any follow up actions. Feedback: {user_feedback}", "tool_call_id": tool_call["id"]})

            else:
                raise ValueError(f"Invalid tool call: {tool_call['name']}")

    # Update the state
    update = {
        "messages": result,
    }

    return Command(goto=goto, update=update)

O resto é igual ao anterior!

In [ ]:
from email_assistant.utils import show_graph

# Conditional edge function
def should_continue(state: State, store: BaseStore) -> Literal["interrupt_handler", END]:
    """Route to tool handler, or end if Done tool called"""
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            if tool_call["name"] == "Done":
                return END
            else:
                return "interrupt_handler"

# Build workflow
agent_builder = StateGraph(State)

# Add nodes - with store parameter
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("interrupt_handler", interrupt_handler)

# Add edges
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    {
        "interrupt_handler": "interrupt_handler",
        END: END,
    },
)

# Compile the agent
response_agent = agent_builder.compile()

# Build overall workflow with store and checkpointer
overall_workflow = (
    StateGraph(State, input=StateInput)
    .add_node(triage_router)
    .add_node(triage_interrupt_handler)
    .add_node("response_agent", response_agent)
    .add_edge(START, "triage_router")
)

email_assistant = overall_workflow.compile()
show_graph(email_assistant)

## Testando o agente com memória

Agora que implementamos memória em nosso assistente de email, vamos testar como o sistema aprende com o feedback do usuário e se adapta ao longo do tempo. Esta seção de testes explora como diferentes tipos de interações do usuário criam atualizações de memória distintas que melhoram o desempenho futuro do assistente.

As principais questões que estamos respondendo através destes testes:
1. Como o sistema captura e armazena as preferências do usuário?
2. Como essas preferências armazenadas afetam decisões futuras?
3. Quais padrões de interação levam a que tipos de atualizações de memória?

Primeiro, vamos construir uma função auxiliar para exibir o conteúdo da memória para que possamos acompanhar como ela evolui ao longo de nossos testes:

In [ ]:
import uuid
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command
from langgraph.store.memory import InMemoryStore

# Helper function to display memory content
def display_memory_content(store, namespace=None):
    # Display current memory content for all namespaces
    print("\n======= CURRENT MEMORY CONTENT =======")
    if namespace:
        memory = store.get(namespace, "user_preferences")
        if memory:
            print(f"\n--- {namespace[1]} ---")
            print(memory.value)
        else:
            print(f"\n--- {namespace[1]} ---")
            print("Nenhuma memória encontrada")
    else:
        for namespace in [
            ("email_assistant", "triage_preferences"),
            ("email_assistant", "response_preferences"),
            ("email_assistant", "cal_preferences"),
            ("email_assistant", "background")
        ]:
            memory = store.get(namespace, "user_preferences")
            if memory:
                print(f"\n--- {namespace[1]} ---")
                print(memory.value)
            else:
                print(f"\n--- {namespace[1]} ---")
                print("Nenhuma memória encontrada")
            print("=======================================\n")

### Aceitar `write_email` e `schedule_meeting`

Nosso primeiro teste examina o que acontece quando um usuário aceita as ações do agente sem modificação. Este caso base nos ajuda a entender como o sistema se comporta quando nenhum feedback é fornecido:

1. Usaremos o mesmo email de planejamento tributário de nossos testes anteriores
2. O sistema o classificará como "RESPOND" e proporá agendar uma reunião
3. Aceitaremos o agendamento da reunião sem mudanças
4. O agente gerará um email confirmando a reunião
5. Aceitaremos o email sem mudanças

Este teste demonstra o comportamento padrão de nosso sistema habilitado para memória. Quando um usuário simplesmente aceita ações propostas, esperamos atualizações mínimas ou nenhuma atualização de memória, já que não há feedback explícito do qual aprender. No entanto, o sistema ainda utilizará a memória existente (se houver) ao gerar suas respostas.

In [ ]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_1 = uuid.uuid4()
thread_config_1 = {"configurable": {"thread_id": thread_id_1}}

# Run the graph until the first interrupt
# Email will be classified as "respond"
# Agent will create a schedule_meeting and write_email tool call
print("Executando o grafo até a primeira interrupção...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_1):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt
display_memory_content(store)

Aceitar a chamada de ferramenta `schedule_meeting`

Ao examinarmos a proposta inicial de `schedule_meeting`, observe como o sistema usa a memória existente para informar suas decisões:

1. As preferências padrão de calendário mostram uma preferência por reuniões de 30 minutos, embora o email solicite 45 minutos
2. O agente ainda propõe uma reunião de 45 minutos, respeitando a solicitação específica do remetente
3. Aceitamos esta proposta sem modificação para ver se a simples aceitação desencadeia alguma atualização de memória

Após executar este passo, verificaremos o conteúdo da memória para confirmar se a aceitação sozinha leva a atualizações de memória. A simples aceitação representa a experiência base do usuário - o sistema funciona conforme pretendido sem exigir ajustes.

In [ ]:
print(f"\nSimulando usuário aceitando a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_1):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Aceitar a chamada de ferramenta `write_email`

Agora aceitaremos o rascunho de email que confirma o agendamento da reunião:

1. O rascunho de email é gerado com conhecimento de nossas preferências de calendário
2. Ele inclui detalhes sobre o horário da reunião, duração e propósito
3. Aceitaremos sem mudanças para completar o caso de teste base

Após aceitar, verificaremos todos os armazenamentos de memória para ver se ocorreram atualizações. Como esperado, simplesmente aceitar as propostas do agente não fornece sinais de aprendizado fortes - não há feedback claro sobre o que o usuário gosta ou não gosta da abordagem do agente.

O link de rastreamento mostra a execução completa do fluxo de trabalho, onde podemos ver que a memória é usada na chamada LLM para geração de resposta, mas nenhuma atualização de memória ocorre, que é o comportamento esperado para simples aceitações.

In [ ]:
print(f"\nSimulando usuário aceitando a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_1):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after accepting the write_email tool call
display_memory_content(store)

Podemos olhar as mensagens completas e o rastreamento: 

https://smith.langchain.com/public/86ff6474-29fe-452e-8829-b05a91b458eb/r

Você notará que a memória é usada na chamada LLM para responder. 

Mas o armazenamento de memória *não* é atualizado, porque não adicionamos nenhum feedback via HITL.

In [ ]:
state = graph.get_state(thread_config_1)
for m in state.values['messages']:
    m.pretty_print()

### Editar `write_email` e `schedule_meeting`

Este teste explora como o sistema aprende a partir de edições diretas às suas ações propostas. Quando usuários modificam as sugestões do agente, isso cria sinais de aprendizado claros e específicos sobre suas preferências:

1. Usaremos o mesmo email de planejamento tributário de antes
2. Quando o agente propor uma reunião de 45 minutos, editaremos para:
   - Mudar a duração para 30 minutos (correspondendo à nossa preferência armazenada)
   - Tornar o assunto mais conciso
3. Quando o agente rascunhar um email, editaremos para ser:
   - Mais curto e menos formal
   - Estruturado de forma diferente

Edições fornecem o feedback mais explícito sobre preferências do usuário, permitindo que o sistema aprenda exatamente quais mudanças são desejadas. Esperamos ver atualizações específicas e direcionadas em nossos armazenamentos de memória que reflitam essas edições.

In [ ]:
# Same email as before
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph with new thread
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_2 = uuid.uuid4()
thread_config_2 = {"configurable": {"thread_id": thread_id_2}}

# Run the graph until the first interrupt - will be classified as "respond" and the agent will create a write_email tool call
print("Executando o grafo até a primeira interrupção...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_2):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt
display_memory_content(store,("email_assistant", "cal_preferences"))

Editar a chamada da ferramenta `schedule_meeting`

Quando editamos a proposta de reunião, estamos fornecendo feedback direto e explícito sobre nossas preferências. Isso cria uma oportunidade significativa de aprendizado para o sistema:

1. O agente inicialmente propõe uma reunião de 45 minutos (a duração solicitada no email)
2. Nós editamos para 30 minutos e simplificamos o assunto de "Tax Planning Strategies Discussion" para "Tax Planning Discussion"
3. Isso cria feedback claro e específico sobre nossas preferências de tempo e convenções de nomenclatura

In [ ]:
# Now simulate user editing the schedule_meeting tool call
print("\nSimulando usuário editando a schedule_meeting chamada de ferramenta...")
edited_schedule_args = {
    "attendees": ["pm@client.com", "lance@company.com"],
    "subject": "Tax Planning Discussion",
    "duration_minutes": 30, # Changed from 45 to 30
    "preferred_day": "2025-04-22",
    "start_time": 14
}
for chunk in graph.stream(Command(resume=[{"type": "edit", "args": {"args": edited_schedule_args}}]), config=thread_config_2):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after editing schedule_meeting
print("\nVerificando memória após editar schedule_meeting:")
display_memory_content(store,("email_assistant", "cal_preferences"))

```
{'preferences': '\n30 minute meetings are preferred, but 15 minute meetings are also acceptable.\n'}
```

```
{'preferences': "30 minute meetings are preferred, but 15 minute meetings are also acceptable.\n\nUser prefers 30 minute meetings over longer durations such as 45 minutes. When scheduling, default to 30 minutes unless otherwise specified. Subject lines should be concise (e.g., 'Tax Planning Discussion' instead of 'Tax Planning Strategies Discussion')."}
```

Olhando para a memória após editar o convite de calendário, podemos ver que ela foi atualizada:

1. O sistema identificou que preferimos reuniões de 30 minutos em vez de durações mais longas
2. Também capturou nossa preferência por assuntos de reunião concisos

O que é particularmente impressionante sobre esta atualização de memória é:
- Ela não apenas registra nossa edição específica, mas generaliza para um padrão de preferência mais amplo
- Preserva todo o conteúdo de memória existente enquanto adiciona a nova informação
- Extrai múltiplos sinais de preferência de uma única interação de edição

Agora, vamos editar o rascunho de email para ver como o sistema captura diferentes tipos de preferências de comunicação:

In [ ]:
display_memory_content(store,("email_assistant", "response_preferences"))
# Now simulate user editing the write_email tool call
print("\nSimulando usuário editando a write_email chamada de ferramenta...")
edited_email_args = {
    "to": "pm@client.com",
    "subject": "Re: Tax season let's schedule call",
    "content": "Thanks! I scheduled a 30-minute call next Thursday at 3:00 PM. Would that work for you?\n\nBest regards,\nLance Martin"
}
for chunk in graph.stream(Command(resume=[{"type": "edit", "args": {"args": edited_email_args}}]), config=thread_config_2):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after editing write_email
print("\nVerificando memória após editar write_email:")
display_memory_content(store,("email_assistant", "response_preferences"))

Nossa edição de email revela capacidades de aprendizado ainda mais sofisticadas:

1. Encurtamos e simplificamos drasticamente o conteúdo do email
2. Mudamos o tom para ser mais casual
3. Adicionamos uma pergunta pedindo confirmação em vez de assumir que o horário funciona
4. Alteramos ligeiramente os detalhes da reunião (dia e horário)

Olhando para a memória atualizada, podemos ver que o sistema extraiu uma percepção chave sobre nosso estilo de comunicação:

```
Ao agendar uma reunião, peça ao destinatário para confirmar se o horário proposto funciona para eles, em vez de assumir e declarar que a reunião já está agendada.
```

Isso demonstra a capacidade do sistema de:
- Analisar nossa edição não apenas em um nível superficial, mas para entender a intenção
- Extrair princípios generalizáveis de exemplos específicos
- Preservar toda a orientação existente enquanto adiciona novas percepções
- Manter a organização e estrutura da memória

Essas atualizações de memória direcionadas e de alta qualidade melhorarão todas as interações futuras sem exigir correções repetidas.

In [ ]:
state = graph.get_state(thread_config_2)
for m in state.values['messages']:
    m.pretty_print()

### Responder (com feedback) `write_email` e `schedule_meeting`

Nosso conjunto final de testes explora o padrão de feedback "resposta" - fornecendo orientação sem editar ou aceitar diretamente. Este mecanismo de feedback conversacional oferece um meio-termo entre aceitação e edição:

1. Primeiro, testaremos feedback para agendamento de reunião solicitando:
   - Duração mais curta (30 minutos em vez de 45)
   - Horários de reunião à tarde (após às 14h)
   
2. Em seguida, testaremos feedback para rascunho de email solicitando:
   - Linguagem mais curta e menos formal
   - Uma declaração de encerramento específica sobre esperar pela reunião
   
3. Finalmente, testaremos feedback para perguntas fornecendo:
   - Uma resposta direta com contexto adicional
   - Preferências específicas (local de brunch, horário)

Esta abordagem de feedback em linguagem natural permite que usuários orientem o assistente sem ter que fazer o trabalho eles mesmos. Esperamos ver atualizações detalhadas de memória que extraiam os princípios gerais de nosso feedback específico.

In [ ]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_5 = uuid.uuid4()
thread_config_5 = {"configurable": {"thread_id": thread_id_5}}

# Run the graph until the first interrupt
# Email will be classified as "respond"
# Agent will create a schedule_meeting and write_email tool call
print("Executando o grafo até a primeira interrupção...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt
display_memory_content(store, ("email_assistant", "cal_preferences"))

Fornecer feedback para a chamada de ferramenta `schedule_meeting`

Em vez de editar diretamente a proposta de reunião ou simplesmente aceitá-la, forneceremos feedback em linguagem natural:

1. Solicitamos uma reunião de 30 minutos em vez de 45 minutos
2. Expressamos uma preferência por reuniões à tarde após às 14h
3. O sistema deve interpretar este feedback e gerar uma nova proposta

Esta abordagem conversacional é frequentemente mais natural e eficiente do que edição direta, especialmente para usuários móveis ou aqueles que preferem dar direção de alto nível em vez de edições detalhadas.

Após fornecer feedback, examinaremos a memória de preferências de calendário para ver como esta orientação em linguagem natural é capturada. Esperamos ver o sistema extrair tanto a duração da reunião quanto as preferências de horário do dia como princípios gerais.

In [ ]:
print(f"\nSimulando usuário fornecendo feedback para a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Please schedule this for 30 minutes instead of 45 minutes, and I prefer afternoon meetings after 2pm."}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after providing feedback for schedule_meeting
print("\nVerificando memória após fornecer feedback para schedule_meeting:")
display_memory_content(store, ("email_assistant", "cal_preferences"))

Nossa verificação de memória após fornecer feedback mostra uma atualização elegantemente simples de preferência de calendário:

```
Reuniões de 30 minutos são preferidas, mas reuniões de 15 minutos também são aceitáveis.
Reuniões à tarde após às 14h são preferidas.
```

O sistema:
1. Capturou ambos os aspectos do nosso feedback (duração e horário do dia)
2. Preservou a preferência existente sobre reuniões de 15 minutos
3. Adicionou nossa preferência por reuniões à tarde após às 14h como uma nova linha
4. Manteve o formato limpo e legível

Este mecanismo de feedback em linguagem natural cria a mesma qualidade de atualizações de memória que a edição direta, mas requer menos esforço do usuário. O sistema é capaz de extrair preferências estruturadas de feedback não estruturado, mostrando sua capacidade de aprender a partir de interações conversacionais.

Vamos aceitar esta proposta de reunião revisada e passar para o rascunho de email:

In [ ]:
print(f"\nSimulando usuário aceitando a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after accepting schedule_meeting after feedback
print("\nVerificando memória após aceitar schedule_meeting após feedback:")
display_memory_content(store, ("email_assistant", "response_preferences"))

Agora fornecer feedback para a chamada de ferramenta `write_email`

Similar ao nosso feedback de reunião, agora forneceremos orientação em linguagem natural para o rascunho de email:

1. Solicitamos linguagem "mais curta e menos formal" - uma preferência de estilo
2. Pedimos uma declaração de encerramento específica sobre esperar pela reunião
3. O sistema deve interpretar esta orientação e reescrever o email adequadamente

Após fornecer este feedback, verificaremos a memória de preferências de resposta para ver como essas preferências de estilo e estrutura são capturadas. Esperamos ver diretrizes generalizáveis sobre brevidade, formalidade e declarações de encerramento adicionadas ao nosso perfil de preferências.

In [ ]:
print(f"\nSimulando usuário fornecendo feedback para a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Shorter and less formal. Include a closing statement about looking forward to the meeting!"}]), config=thread_config_5):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after providing feedback for write_email
print("\nVerificando memória após fornecer feedback para write_email:")
display_memory_content(store, ("email_assistant", "response_preferences"))

A atualização de memória após nosso feedback de email mostra aprendizado altamente sofisticado sobre preferências tanto de agendamento de reunião quanto de escrita de email:

1. O sistema adicionou uma seção completamente nova às preferências de resposta intitulada "Ao escrever respostas de email" com duas preferências principais:
   - "Favoreça linguagem mais curta e menos formal quando possível, a menos que o contexto exija formalidade"
   - "Inclua uma declaração de encerramento expressando que você espera pela reunião ou conversa ao confirmar compromissos"

2. Também adicionou um novo ponto à seção "Ao responder a solicitações de agendamento de reunião":
   - "Ao agendar reuniões, prefira horários à tarde após às 14h quando possível, e padrão para durações de 30 minutos a menos que especificado de outra forma"

Isso demonstra a capacidade do sistema de:
- Organizar preferências aprendidas em categorias apropriadas
- Extrair múltiplas percepções de uma única instância de feedback
- Aplicar preferências de reunião tanto em contextos de calendário quanto de email
- Capturar nuance com qualificadores apropriados ("quando possível", "a menos que especificado de outra forma")
- Manter a estrutura hierárquica da memória

O email resultante mostra todas essas preferências aplicadas: é mais curto, menos formal, inclui uma declaração de encerramento sobre esperar pela conversa, e referencia corretamente o horário de reunião de 30 minutos.

In [ ]:
print(f"\nSimulando usuário aceitando a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

# Check memory after accepting write_email after feedback
print("\nVerificando memória após aceitar write_email após feedback:")
display_memory_content(store, ("email_assistant", "response_preferences"))

Olhar o histórico completo de mensagens.

In [ ]:
state = graph.get_state(thread_config_5)
for m in state.values['messages']:
    m.pretty_print()

## Local Deployment

You can find this graph with memory integration in the `src/email_assistant` directory:

* `src/email_assistant/email_assistant_hitl_memory.py`

Email to test: 
```
{
  "author": "Alice Smith <alice.smith@company.com>",
  "to": "John Doe <john.doe@company.com>",
  "subject": "Quick question about API documentation",
  "email_thread": "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
}
```

As before, if you go to [dev.agentinbox.ai](https://dev.agentinbox.ai/), you can easily connect to the graph:

   * Graph name: the name from the `langgraph.json` file (`email_assistant_hitl_memory`)
   * Graph URL: `http://127.0.0.1:2024/`

![inbox](img/agent-inbox-edit.png)

The Memory tab in LangGraph Studio offers a real-time view of how your preferences are being captured and updated with each interaction:

![studio-img](img/memory-studio.png)

Through continued use, the system becomes increasingly personalized:
- It learns which emails you want to respond to, be notified about, or ignore
- It adapts to your communication style preferences
- It remembers your scheduling preferences
- It refines its understanding with each interaction

This combination of HITL and memory creates a system that balances automation with control - handling routine tasks automatically while learning from your feedback to become more aligned with your preferences over time.

## Hosted Deployment with Gmail Tools

If you want to actually run this on your own email, you can deploy the graph with Gmail tools. 

Set up your Gmail credentials [following here](https://github.com/langchain-ai/agents-from-scratch/blob/main/src/email_assistant/tools/gmail/README.md).

There is a graph set up with Gmail tools:

```shell
python src/email_assistant/email_assistant_hitl_memory_gmail.py
```

[One of the deployment options is `hosted`](https://langchain-ai.github.io/langgraph/tutorials/deployment/#other-deployment-options), and you can simply connect the deployed graph URL to the Agent Inbox as done with the local deployment.

## Improving Memory 

Our current memory schema and updating is extremely simple: 

* Our schema is a string
* We always overwrite the existing memory with a new string
 
The store can be easily [configured for semantic search](https://langchain-ai.github.io/langgraph/cloud/deployment/semantic_search/) over a collection of memories. 

Also consider using [LangMem](https://langchain-ai.github.io/langmem/) for more advanced memory management.